<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')
TEST_PATH  = os.path.join(DATA_PATH, 'test.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [4]:
df = df.sample(False, 0.5)

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

In [6]:
pandas_df = df.limit(20).toPandas()

pandas_df.loc[:, :'_c13']

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13
0,0,8.0,0,15.0,20.0,115.0,24.0,8.0,23,24.0,2.0,2.0,NaN,20.0
1,1,1.0,0,31.0,9.0,226.0,22.0,32.0,13,177.0,1.0,7.0,0.0,11.0
2,0,0.0,76,3.0,NaN,5029.0,NaN,NaN,16,NaN,0.0,NaN,NaN,NaN
3,1,1.0,5,33.0,6.0,375.0,74.0,3.0,46,258.0,1.0,2.0,NaN,68.0
4,1,2.0,30,27.0,2.0,0.0,0.0,2.0,2,2.0,1.0,1.0,NaN,0.0
5,0,NaN,19,31.0,8.0,7130.0,126.0,1.0,20,116.0,NaN,1.0,NaN,8.0
6,0,NaN,2242,13.0,7.0,11786.0,63.0,29.0,13,87.0,NaN,1.0,NaN,13.0
7,0,NaN,70,7.0,2.0,6518.0,26.0,23.0,2,12.0,NaN,2.0,NaN,2.0
8,1,NaN,0,1.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
9,0,NaN,0,NaN,3.0,1992.0,5.0,6.0,5,5.0,NaN,1.0,NaN,3.0


In [7]:
pandas_df.loc[:, '_c14':'_c26']

,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25,_c26
0,5a9ed9b0,c66fca21,78171040,373c404a,25c83c98,None,8ff6f5af,0b153874,a73ee510,5ba575e7,b5a9f90e,6766a7f0,949ea585
1,5a9ed9b0,942f9a8d,013bceb0,136ff514,25c83c98,6f6d9be8,3f4ec687,7b6fecd5,a73ee510,726f00fd,c4adf918,65b6ef3b,85dbe138
2,05db9164,421b43cd,889a923c,29998ed1,384874ce,fe6b92e5,52283d1c,37e4aa92,a73ee510,03e48276,e51ddf94,6aaba33c,3516f6e6
3,05db9164,bce95927,71da6210,13508380,25c83c98,7e0ccccf,86aa3994,0b153874,a73ee510,39cda501,61593534,944ea880,9259d03d
4,05db9164,c1384774,d4bef5d2,ebc3fea2,25c83c98,fbad5c96,81bb0302,0b153874,a73ee510,70962768,b7094596,e52ba8a9,1f9d2c38
5,05db9164,39dfaa0d,8f9df49c,c0c8ad07,25c83c98,None,753aa291,1f89b562,a73ee510,3b08e48b,d20ffd8f,dab8680a,6619af2b
6,be589b51,4c2bc594,d032c263,c18be181,25c83c98,7e0ccccf,7227c706,1f89b562,a73ee510,305a0646,9625b211,dfbb09fb,dccbd94b
7,05db9164,80e26c9b,0925f7d4,65ee5fae,25c83c98,7e0ccccf,08e57a96,0b153874,a73ee510,5ba575e7,7c430b79,d43bc772,7f0d7407
8,05db9164,38a947a1,e3818eb2,a4456f7e,25c83c98,fbad5c96,88002ee1,5b392875,7cc72ec2,3b08e48b,f1b78ab4,3bec5d45,6e5da64f
9,68fd1e64,78ccd99e,24ef584c,c8167e68,4cf72387,None,1b76cf1e,1f89b562,a73ee510,b43f9937,0d8e34fa,e9e2c23f,e21429e2


In [8]:
pandas_df.loc[:, '_c27':'_c39']

,_c27,_c28,_c29,_c30,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39
0,1adce6ef,8736735c,59974c9c,8efede7f,1304f63b,21ddcdc9,b1252a9d,07b2853e,None,32c7478e,94bde4f2,010f6491,09b76f8d
1,1adce6ef,ae97ecc3,a55f20ce,3486227d,1f868fdd,cd11c728,a458ea53,00680113,None,32c7478e,ad4c56a0,e8b83407,8270b5de
2,b28479f6,e1ac77f7,b041b04a,d4bb7bd8,2804effd,None,None,723b4dfd,None,3a171ecb,b34f3128,None,None
3,07d13a8f,fec218c0,a13376db,e5ba7672,04d863d5,54e53a39,b1252a9d,f1e1f2b3,None,423fab69,45ab94c8,e8b83407,c84c4aec
4,b28479f6,916e9a2c,0982799e,07c540c4,8e8b535e,21ddcdc9,b1252a9d,11da5050,ad3062eb,32c7478e,9d214089,ea9a246c,5d3f5a67
5,b28479f6,2223bbe1,ceda4ce2,776ce399,df4fffb7,21ddcdc9,5840adea,09f172ad,None,3a171ecb,c8a524a7,010f6491,4e7af834
6,8ceecbc8,7ac43a46,84898b2a,e5ba7672,bc48b783,None,None,0014c32a,ad3062eb,55dd3565,3b183c5c,None,None
7,07d13a8f,f3635baf,3a4e2f87,8efede7f,f54016b9,21ddcdc9,b1252a9d,87a78db1,8ec974f4,3a171ecb,7cadb3d7,e8b83407,705ee187
8,1adce6ef,3e704e66,72d1790f,2005abd1,f03e8b05,None,None,950d91c1,None,55dd3565,2f7e98de,None,None
9,051219e6,9917ad07,f95cc162,1e88c74f,e7e991cb,21ddcdc9,a458ea53,f0db6227,None,32c7478e,1f163fc7,ea9a246c,8c532d04


Keeping only first two categorical features for simplicity

In [9]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [10]:
df = df.fillna(0, subset=num_columns) # TODO: try different default value

In [11]:
df.limit(20).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,id
0,0,8,0,15,20,115,24,8,23,24,...,1304f63b,21ddcdc9,b1252a9d,07b2853e,None,32c7478e,94bde4f2,010f6491,09b76f8d,39
1,1,1,0,31,9,226,22,32,13,177,...,1f868fdd,cd11c728,a458ea53,00680113,None,32c7478e,ad4c56a0,e8b83407,8270b5de,121
2,0,0,76,3,0,5029,0,0,16,0,...,2804effd,None,None,723b4dfd,None,3a171ecb,b34f3128,None,None,206
3,1,1,5,33,6,375,74,3,46,258,...,04d863d5,54e53a39,b1252a9d,f1e1f2b3,None,423fab69,45ab94c8,e8b83407,c84c4aec,212
4,1,2,30,27,2,0,0,2,2,2,...,8e8b535e,21ddcdc9,b1252a9d,11da5050,ad3062eb,32c7478e,9d214089,ea9a246c,5d3f5a67,249
5,0,0,19,31,8,7130,126,1,20,116,...,df4fffb7,21ddcdc9,5840adea,09f172ad,None,3a171ecb,c8a524a7,010f6491,4e7af834,252
6,0,0,2242,13,7,11786,63,29,13,87,...,bc48b783,None,None,0014c32a,ad3062eb,55dd3565,3b183c5c,None,None,268
7,0,0,70,7,2,6518,26,23,2,12,...,f54016b9,21ddcdc9,b1252a9d,87a78db1,8ec974f4,3a171ecb,7cadb3d7,e8b83407,705ee187,297
8,1,0,0,1,0,0,0,0,1,0,...,f03e8b05,None,None,950d91c1,None,55dd3565,2f7e98de,None,None,302
9,0,0,0,0,3,1992,5,6,5,5,...,e7e991cb,21ddcdc9,a458ea53,f0db6227,None,32c7478e,1f163fc7,ea9a246c,8c532d04,376


## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm that can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm that can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import  MinMaxScaler, StringIndexer, OneHotEncoderEstimator, VectorAssembler


# in proccess
columns = cat_columns[:2]
columns_indexed = [col + '_indexed' for col in columns]
columns_vectorized = [col + '_vec' for col in columns]

vector_assembler = VectorAssembler(inputCols=num_columns, outputCol="num_features")
string_indexers = [StringIndexer(inputCol=col, outputCol=col + '_indexed', handleInvalid="keep") for col, col_ind in zip(columns, columns_indexed)]
one_hot_encoder = OneHotEncoderEstimator(inputCols=columns_indexed, outputCols=columns_vectorized)
scaler = MinMaxScaler(inputCol="num_features", outputCol="scaled_features")
final_assembler = VectorAssembler(inputCols=['scaled_features'] + columns_vectorized, outputCol="features")

pipeline = Pipeline(stages=string_indexers + [one_hot_encoder] + [vector_assembler] + [scaler] + [final_assembler])

### Fit and save pipeline

In [13]:
import os
import shutil
pipeline_model = pipeline.fit(df)

path = "transforming_pipeline"
if os.path.exists(path):
    shutil.rmtree(path)
pipeline_model.save(path)
path

'transforming_pipeline'

### Load fitted pipeline

In [14]:
from pyspark.ml import PipelineModel, Pipeline

pipeline_model = PipelineModel.load(path)

### Transform dataset using pipeline

In [15]:
df = pipeline_model.transform(df).select('_c0', 'features', 'id')

df.limit(10).toPandas()

,_c0,features,id
0,0,"(0.0013852813852813853, 9.062896501721951e-05,...",39
1,1,"(0.00017316017316017316, 9.062896501721951e-05...",121
2,0,"(0.0, 0.0035345296356715608, 4.577706569008926...",206
3,1,"(0.00017316017316017316, 0.0003172013775602683...",212
4,1,"(0.0003463203463203463, 0.0014500634402755121,...",249
5,0,"(0.0, 0.0009516041326808048, 0.000473029678797...",252
6,0,"(0.0, 0.10168569874932028, 0.00019836728465705...",268
7,0,"(0.0, 0.0032626427406199023, 0.000106813153276...",297
8,1,"(0.0, 9.062896501721951e-05, 1.525902189669642...",302
9,0,"(0.0, 9.062896501721951e-05, 0.0, 0.0054644808...",376


### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [16]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import lit

def split_by_col(df, split_col, part_fractions):
    """
    df - DataFrame
    split_col - total order column
    part_fractions - fractions of the resulting parts
    """
    
    window = Window.orderBy(split_col).partitionBy('const')
    ranks = df \
        .select(split_col) \
        .distinct() \
        .withColumn('const', F.lit(0)) \
        .withColumn('percent_rank', F.percent_rank().over(window)) \
        .select(split_col, 'percent_rank')

    df = df.join(ranks, split_col)
    
    parts = []
    s = 0
    for part_fraction in part_fractions:
        l = s
        r = s + part_fraction
        part = df \
            .filter((l <= F.col('percent_rank')) & (F.col('percent_rank') < r)) \
            .drop('percent_rank')
        
        parts.append(part)
        s = r
        
    return parts

In [17]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [18]:
N = df.count()
N

1831028

In [19]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999997815434827, 0.10000010922825867, 0.09999956308696535)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [20]:
%%time

from pyspark.ml.classification import LogisticRegression

lr_model = LogisticRegression(featuresCol = 'features', labelCol = '_c0', maxIter=10).fit(train_df)

CPU times: user 23.7 ms, sys: 6.63 ms, total: 30.4 ms
Wall time: 42.9 s


In [26]:
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

get_second_element = F.udf(lambda v:float(v[1]),FloatType())

df_predictions = lr_model.transform(val_df)
df_predictions \
  .withColumn('proba', get_second_element(F.col('probability'))) \
  .first()

Row(id=455266587038, _c0=0, features=SparseVector(1941, {1: 0.0001, 2: 0.0, 3: 0.0128, 7: 0.003, 8: 0.0003, 12: 0.0015, 18: 1.0, 1514: 1.0}), rawPrediction=DenseVector([1.4334, -1.4334]), probability=DenseVector([0.8074, 0.1926]), prediction=0.0, proba=0.1925620287656784)

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [ ]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics


def rocauc(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass


def logloss(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass


def ne(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass

In [ ]:
rocauc(lr_model, val_df)

In [ ]:
ne(lr_model, val_df)

In [ ]:
rocauc(lr_model, test_df)

In [ ]:
ne(lr_model, test_df)

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [32]:
test_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

test_df = test_df.fillna(0, subset=num_columns) 

In [33]:
test_df.limit(10).toPandas()

,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,...,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,id
0,0,19,2,4,4576,6,6,5,15,0,...,43de85d3,None,None,b64021bd,None,32c7478e,f1a27f66,None,None,566935904713
1,0,1,1,0,5688,0,0,2,10,0,...,e7e991cb,efa3470f,a458ea53,6ef75f1d,78e2e389,32c7478e,f53ea242,cb079c2d,e9b68fcc,566935904715
2,0,445,2,2,8579,26,1,2,26,0,...,e5f8f18f,None,None,f3ddd519,ad3062eb,32c7478e,b34f3128,None,None,566935904727
3,0,172,7,1,2008,143,24,28,430,0,...,eef7297e,None,None,8ae05402,ad3062eb,423fab69,8d4a9014,None,None,566935904737
4,0,11,4,4,14,0,0,4,6,0,...,7181ccc8,None,None,2265e99d,None,32c7478e,5dc43b96,None,None,566935904741
5,1,0,19,27,108,33,14,33,245,1,...,47e4d79e,21ddcdc9,b1252a9d,411e9d2a,None,423fab69,ab246148,e8b83407,21db0a46,566935904745
6,7,2,2,6,296,13,27,9,227,1,...,e589f353,None,None,805e1e60,None,55dd3565,c5cbfccf,None,None,566935904752
7,0,0,210,46,16102,0,0,46,52,0,...,891589e7,5b885066,a458ea53,13e6792a,c9d4222a,93bad2c0,4001cb4f,ea9a246c,aee7bde0,566935904759
8,0,2,1,1,17621,40,3,5,66,0,...,f8286ab3,None,None,dcedd4b3,ad3062eb,32c7478e,b6654c21,None,None,566935904767
9,0,56,0,0,9794,0,0,30,0,0,...,e4ca448c,None,None,d9bd2fe2,None,bcdee96c,9117a34a,None,None,566935904768


In [34]:
test_df = pipeline_model.transform(test_df).select('features', 'id')

In [35]:
test_predictions = lr_model.transform(test_df)
test_predictions = test_predictions \
  .withColumn('proba', get_second_element(F.col('probability'))) \
  .select('id', 'proba')

test_predictions.first()

Row(id=566935904713, proba=0.2513972818851471)

In [42]:
path = os.path.join(DATA_PATH, 'submition.csv')
test_predictions.toPandas().to_csv(path, index=False)